# Python II

1. Exception Handling
2. File Handling
3. Object Oriented Programming I
4. Functional Programming
5. Threading and Processing $\longleftarrow$

---

## Threading and Processing

- **Threads:** Threads are the smallest units of execution within a process. Multiple threads can exist within a single process, and they share the same memory space. Threads are lightweight and used for concurrent execution of tasks.

- **Processors:** Processors, also known as CPUs (Central Processing Units), are the hardware components of a computer that execute instructions. Modern computers often have multiple processor cores, allowing for parallel execution of tasks.

**Multithreading in Python:**

- **Multithreading:** Multithreading is a programming technique that uses multiple threads within a single process to perform tasks concurrently. In Python, the Global Interpreter Lock (GIL) limits the true parallelism of threads. However, multithreading can still be useful for tasks that involve I/O-bound operations like file I/O or network requests.

- **Use Cases:** Multithreading is suitable for tasks where the program spends a significant amount of time waiting for external resources (e.g., file reads, network requests). It can be used to improve the responsiveness and performance of I/O-bound applications.

**Multiprocessing in Python:**

- **Multiprocessing:** Multiprocessing is a technique that utilizes multiple processes, each with its own memory space and Python interpreter. This allows for true parallelism, as the GIL doesn't affect processes. Multiprocessing is used for CPU-bound tasks that involve extensive computations.

- **Use Cases:** Multiprocessing is suitable for computationally intensive tasks like data processing, numerical simulations, and running CPU-bound algorithms in parallel. It can take full advantage of multi-core processors for improved performance.

In summary, multithreading is used for concurrent execution of tasks within a single process and is suitable for I/O-bound operations. Multiprocessing, on the other hand, uses multiple processes for parallel execution and is ideal for CPU-bound tasks. The choice between multithreading and multiprocessing depends on the nature of the task and the hardware available. Python provides libraries like `threading` for multithreading and `multiprocessing` for multiprocessing to facilitate concurrent and parallel programming.

![Concurrency vs Parallelism](https://i.ytimg.com/vi/RlM9AfWf1WU/sddefault.jpg)

---

### Multithreading

Multithreading allows multiple threads (smaller units of a process) to run concurrently. 

In [2]:
import threading
import time

#### Normal Threads

The main program waits for all non-daemon threads to finish before it exits. In other words, a normal thread will keep the program running until it finishes its task. These threads are intended to complete their execution, even if the main program finishes.

In [3]:
print(threading.active_count())  # returns no. of active threads
print(threading.enumerate())     # returns descp. of active threads

6
[<_MainThread(MainThread, started 20324)>, <Thread(IOPub, started daemon 22340)>, <Heartbeat(Heartbeat, started daemon 8560)>, <ControlThread(Control, started daemon 11516)>, <HistorySavingThread(IPythonHistorySavingThread, started 22220)>, <ParentPollerWindows(Thread-4, started daemon 22264)>]


In [1]:
def task(name, delay):
    print(f"Task {name} starting")
    time.sleep(delay)
    print(f"Task {name} completed")

In [4]:
# without multithreading
start = time.time()

task("A", 2)
task("B", 2)

print(f"Total time taken: {time.time() - start:.2f} seconds")

Task A starting
Task A completed
Task B starting
Task B completed
Total time taken: 4.00 seconds


In [5]:
# with multithreading
start = time.time()

# create threads
thread1 = threading.Thread(target=task, args=("A", 2))
thread2 = threading.Thread(target=task, args=("B", 2))

# start threads
thread1.start()  # func is called
thread2.start()  # func is called

print(threading.active_count())  # +2
print(threading.enumerate())

# wait for both threads to finish
thread1.join()
thread2.join()

print(f"Total time taken: {time.time() - start:.2f} seconds")

Task A starting
Task B starting
8
[<_MainThread(MainThread, started 20324)>, <Thread(IOPub, started daemon 22340)>, <Heartbeat(Heartbeat, started daemon 8560)>, <ControlThread(Control, started daemon 11516)>, <HistorySavingThread(IPythonHistorySavingThread, started 22220)>, <ParentPollerWindows(Thread-4, started daemon 22264)>, <Thread(Thread-5 (task), started 7888)>, <Thread(Thread-6 (task), started 8952)>]
Task A completedTask B completed

Total time taken: 2.01 seconds


**Note:** Not calling `join()` means the main thread/process won’t wait for child threads/processes to complete. However, these threads/processes will still finish unless the main program exits/ends.

#### Daemon Threads

Daemon threads run in the background and are typically used for tasks that should not block the main program from exiting. They are meant for tasks that are not essential to the program and can be safely terminated when the main program ends.

In [6]:
def background_task():
    while True:
        print("Background task is running...")
        time.sleep(1)

**Note:** Don't run in `.ipynb` notebooks.

In [ ]:
# create a daemon thread
daemon_thread = threading.Thread(target=background_task, daemon=True)
# daemon_thread.daemon = True  
daemon_thread.start()

# main thread sleeps for 3 seconds and exits
time.sleep(3)
print("Main program exits")

**Note:** The above is an example of `daemon threads`. They run in the background and are terminated when the main program exits. They are useful for tasks that don't need to complete before the program ends.

**Note:** Normal threads will block the program from exiting until they complete, while daemon threads will not block the program from exiting, and they will be terminated when the program ends.

---

### Multiprocessing

Multiprocessing, unlike threads, which run within the same process, it creates separate processes, each with its own memory space. They achieve true parallelism. Each process runs independently on a separate CPU core therefore memory can't be shared.

In [9]:
import multiprocessing

In [10]:
multiprocessing.cpu_count()  # returns no. of cpu cores

12

In [16]:
# without multiprocessing
def compute_square(num):
    result = num * num
    time.sleep(2)  # simulating a CPU-bound task
    print(f"Square of {num}: {result}")


start = time.time()
compute_square(3)
compute_square(4)

print(f"Total time taken: {time.time() - start:.2f} seconds")

Square of 3: 9
Square of 4: 16
Total time taken: 4.00 seconds


**Note:** In JupyterLab, using multiprocessing may not work as expected due to the way Jupyter processes are managed. In such cases, it's recommended to use multiprocessing in a regular Python script or a different environment. 

In [18]:
# with multiprocessing
def compute_square(num):
    result = num * num
    time.sleep(2)  # simulating a CPU-bound task
    print(f"Square of {num}: {result}")


if __name__ == '__main__':
    start = time.time()

    # Create processes
    process1 = multiprocessing.Process(target=compute_square, args=(3,))
    process2 = multiprocessing.Process(target=compute_square, args=(4,))

    # Start processes
    process1.start()
    process2.start()

    # Wait for processes to complete
    process1.join()
    process2.join()

    print(f"Total time taken: {time.time() - start:.2f} seconds")

Total time taken: 0.14 seconds


- Without `if __name__ == "__main__":` New processes will continuously re-import the script, which can lead to recursion or errors (especially on platforms like Windows).

- With `if __name__ == "__main__":` The main script runs safely, and new processes are spawned without issues.

---